## Day 10
https://adventofcode.com/2025/day/10

### Part 1

In [1]:
import numpy as np

def read_input_10(filename):
    '''
    Represent target lights as bit sequence, as well as button presses.
    A button press operation is effectively a bitwise OR operation.
    For Part 2, saving joltage as numpy array to ease additions.
    '''
    with open(filename) as f:
        machines = []
        for l in f.readlines():
            m = l.strip("\n").split(" ")
            lights = m[0][1:-1].replace(".","0").replace("#","1")
            joltage = np.array([int(n) for n in m[-1][1:-1].split(",")])
            buttons = []
            for b in m[1:-1]:
                button = len(lights)*["0"]
                for i in [int(k) for k in b[1:-1].split(",")]:
                    button[i] = "1"
                buttons.append("".join(button))
            machine = (int(lights,base=2), [int(b,base=2) for b in buttons], joltage)
            machines.append(machine)
    return machines

In [2]:
from collections import deque

def min_xor_steps(target, masks):
    '''
    BFS search for smaller number of presses
    target = final light configuration, as interger bit sequence
    masks = collection of button presses, as interger bit sequence  
    '''
    start = 0
    queue = deque([(start, 0)]) # (value, steps)
    visited = {start}

    while queue:
        value, steps = queue.popleft()
        if value == target:
            return steps

        for m in masks:
            new_val = value ^ m
            if new_val not in visited:
                visited.add(new_val)
                queue.append((new_val, steps + 1))
    
    return None # unreachable

In [3]:
def part1(filename):
    machines = read_input_10(filename)
    presses = 0 
    for machine in machines:
        lights, buttons, _ = machine
        presses += min_xor_steps(lights, buttons)
    return presses

In [4]:
print("Test 1:", part1("examples/example10.txt"))
print("Part 1:", part1("AOC2025inputs/input10.txt"))

Test 1: 7
Part 1: 500


### Part 2

Trying to adapt BFS solution to Part 1, but it only works for the example, despite the attempt to prune the states

In [5]:
def min_add_steps(target, increments):
    '''
    BFS search for smaller number of presses for Part 2
    target = final joltage configuration, as numpy array
    increments = collection of button presses, as numpy array
    It takes forever, despite attempt to prune the states...
    '''
    start = np.array([0]*len(target))
    queue = deque([(start, 0)]) # (value, steps)
    visited = {hash(start.tobytes())}

    while queue:
        value, steps = queue.popleft()
        if all(value == target):
            return steps

        for inc in increments:
            new_val = value + inc

            # Prune: skip if any counter exceeds target
            if any(new_val[i] > target[i] for i in range(len(target))):
                continue
            
            new_val_hash = hash(new_val.tobytes())
            if new_val_hash not in visited:
                visited.add(new_val_hash)
                queue.append((new_val, steps + 1))
    
    return None # unreachable

In [6]:
def part2bfs(filename):
    machines = read_input_10(filename)
    presses = 0 
    for i,machine in enumerate(machines):
        #print(f"{i+1} / {len(machines)} ...")
        _, buttons, joltage = machine
        # convert buttons values from Part 1 into presses numpy arrays
        increments = []
        for b in buttons:
            increments.append(np.array([int(n) for n in bin(b)[2:].zfill(len(joltage))])) 
        presses += min_add_steps(joltage, increments)
    return presses

In [8]:
print("Test 2:", part2bfs("examples/example10.txt"))

Test 2: 33


In [9]:
# This still takes forever, despite the state pruning attempt. Moving to a different approach!
#print("Part 2:", part2bfs("AOC2025inputs/input10.txt"))

### Linear algebra?

This must be solvable matematically, since basically I want to solve a linear system:

$\sum_i a_i B_i = J$

where $J$ is the joltage vector, and $B_i$ are the button press vectors; I want to find that solution so that $\sum_i a_i$ is minimal. This can be rewritten in matricial form as:

$B a = J$

Where **$B$ is the matrix of the button press vectors arranged by columns, and $a$ is the vector of number of button presses**.

In [94]:
machines = read_input_10("examples/example10.txt")
_, buttons, joltage = machines[0]

# Convert button values to increment arrays
increments = []
for b in buttons:
    increments.append(np.array([int(n) for n in bin(b)[2:].zfill(len(joltage))])) 

# Write increment in matricial form
B = np.array(increments).T

# Test example solution from puzzle text
# "One way to do this is by pressing (3) once, (1,3) three times, (2,3) three times, (0,2) once, and (0,1) twice. ]"
a = [ 1, 3, 0, 3, 1, 2]

B @ a, joltage

(array([3, 5, 4, 7]), array([3, 5, 4, 7]))

On the other hand, the problem is **not really solvable with _simple_ linear algebra** because I need **integer solutions** (not real numbers), **positive solutions** (can't press buttons negative times), and I want to **minimize the sum**.

**I could use `scipy`: this is a minimisation problem with a constraint...**

In [133]:
from scipy.optimize import milp, LinearConstraint, Bounds
import math

def min_add_steps_scipy(target, increments):
    '''
    Use scipy's MILP (Mixed-Integer Linear Programming) solver to find minimum button presses
    '''
    n_buttons = len(increments)
    n_counters = len(target)
    
    # Objective: minimize sum of all button presses 
    # $\sum_i c_i * a_i, c_i = [1,1,...,1]$
    c = np.ones(n_buttons) # coefficient of linear combination of button press vector components
    
    # Constraint matrix: each row is a counter, each column is a button
    # B * a = target
    # a is the vector of the number of times each button is pressed
    # B is the matrix of the button presses, aligned by columns
    
    # Generate A matrix from increments list by convert list of arrays to 2D array and transposing
    B = np.array(increments).T
    
    # Equality constraints: B * a == target
    constraints = LinearConstraint(B, lb=target, ub=target)
    
    # Bounds: all variables must be non-negative integers
    bounds = Bounds(lb=0, ub=np.inf)
    
    # Specify that all variables are integers
    # integrality parameter tells scipy which variables must be integers vs which can be real numbers
    integrality = np.ones(n_buttons) # 1 means integer variable
    
    # Solve
    result = milp(c=c, constraints=constraints, bounds=bounds, integrality=integrality)
    
    if result.success:
        #return int(sum(result.x)) # Part 2 result wrong by 1 because of rounding issues when casting to int! 
                                   # int(np.float64(114.99999999999999)) = 114
                                   # math.ceil(np.float64(114.99999999999999)) = 115
        return math.ceil(sum(result.x))
    else:
        return None # No solution found

In [134]:
def part2(filename):
    machines = read_input_10(filename)
    presses = 0
    for machine in machines:
        _, buttons, joltage = machine
        # Convert button values to increment arrays
        increments = []
        for b in buttons:
            increments.append(np.array([int(n) for n in bin(b)[2:].zfill(len(joltage))])) 
        presses += min_add_steps_scipy(joltage, increments) )
    return presses

In [135]:
print("Test 2:", part2("examples/example10.txt"))

Test 2: [10, 12, 11]


In [127]:
print("Part 2:", part2("AOC2025inputs/input10.txt"))

Part 2: [np.float64(66.0), np.float64(31.0), np.float64(134.0), np.float64(46.0), np.float64(72.0), np.float64(225.0), np.float64(114.99999999999999), np.float64(28.0), np.float64(60.0), np.float64(32.0), np.float64(60.0), np.float64(60.0), np.float64(321.0), np.float64(39.0), np.float64(76.0), np.float64(50.0), np.float64(55.0), np.float64(93.0), np.float64(26.0), np.float64(68.0), np.float64(81.0), np.float64(210.0), np.float64(256.0), np.float64(43.0), np.float64(50.0), np.float64(177.0), np.float64(54.0), np.float64(45.0), np.float64(32.0), np.float64(292.0), np.float64(49.0), np.float64(49.0), np.float64(49.0), np.float64(7.0), np.float64(86.0), np.float64(235.0), np.float64(101.0), np.float64(79.0), np.float64(232.0), np.float64(93.0), np.float64(204.0), np.float64(77.0), np.float64(71.0), np.float64(52.0), np.float64(31.0), np.float64(288.0), np.float64(259.0), np.float64(61.0), np.float64(42.0), np.float64(58.0), np.float64(97.0), np.float64(64.0), np.float64(29.0), np.float64(

In [128]:
# 19762  is too low

In [129]:
s = np.float64(114.99999999999999)

In [131]:
i

114

In [132]:
math.ceil(np.float64(114.99999999999999))

115